In [1]:
import os
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


In [2]:
DATASET_PATH = "VGGFace2_balanced_900_albumentations"
IMAGE_SIZE = (224, 224)   # ResNet input
BATCH_SIZE = 32
NUM_CLASSES = 15


In [3]:
# Data augmentation

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.02) 
])


# Load class names
class_names = sorted([d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))])
class_to_int = {name: i for i, name in enumerate(class_names)}

# Process image function
def process_path(file_path):
    class_name = tf.strings.split(file_path, os.path.sep)[-2]
    label = tf.py_function(lambda x: class_to_int[x.numpy().decode()], [class_name], tf.int32)
    label.set_shape([])
    img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    return img, label

# Configure dataset performance
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=3000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

# Get all image paths
all_image_paths = [os.path.join(dp, f) 
                   for dp, dn, fn in os.walk(os.path.expanduser(DATASET_PATH)) 
                   for f in fn]
random.shuffle(all_image_paths)

# Split train/val/test
total_images = len(all_image_paths)
train_size = int(0.7 * total_images)
val_size = int(0.15 * total_images)
train_paths = all_image_paths[:train_size]
val_paths = all_image_paths[train_size:train_size + val_size]
test_paths = all_image_paths[train_size + val_size:]

# Build datasets
train_ds = configure_for_performance(tf.data.Dataset.from_tensor_slices(train_paths).map(process_path, num_parallel_calls=tf.data.AUTOTUNE))
val_ds = configure_for_performance(tf.data.Dataset.from_tensor_slices(val_paths).map(process_path, num_parallel_calls=tf.data.AUTOTUNE))
test_ds = configure_for_performance(tf.data.Dataset.from_tensor_slices(test_paths).map(process_path, num_parallel_calls=tf.data.AUTOTUNE))




In [4]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(
    weights="imagenet",
    include_top=False,  
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze convolutional layers


In [5]:
from tensorflow.keras.applications.resnet50 import preprocess_input

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.layers.Lambda(preprocess_input)(x)  

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,116,111 (92.00 MB)

 Trainable params: 528,399 (2.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, min_lr=1e-7, verbose=1)
checkpoint = ModelCheckpoint('resnet_transfer.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)


In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[checkpoint, early_stop, reduce_lr]
)


Epoch 1/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2037 - loss: 2.4191
Epoch 1: val_accuracy improved from None to 0.55407, saving model to resnet_transfer.keras
296/296 ━━━━━━━━━━━━━━━━━━━━ 576s 2s/step - accuracy: 0.3003 - loss: 2.0460 - val_accuracy: 0.5541 - val_loss: 1.4168 - learning_rate: 1.0000e-04
Epoch 2/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4892 - loss: 1.4625
Epoch 2: val_accuracy improved from 0.55407 to 0.65086, saving model to resnet_transfer.keras
296/296 ━━━━━━━━━━━━━━━━━━━━ 537s 2s/step - accuracy: 0.5244 - loss: 1.3823 - val_accuracy: 0.6509 - val_loss: 1.1062 - learning_rate: 1.0000e-04
Epoch 3/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6049 - loss: 1.1613
Epoch 3: val_accuracy improved from 0.65086 to 0.71951, saving model to resnet_transfer.keras
296/296 ━━━━━━━━━━━━━━━━━━━━ 524s 2s/step - accuracy: 0.6250 - loss: 1.1213 - val_accuracy: 0.7195 - val_loss: 0.9104 - learning_rate: 1.0000e-04
Epoch 4/20
296/296 ━━━━━━━━━

In [12]:
model.save_weights("resnet_transfer_weights.weights.h5")
